# simpy model results collection - **advanced** methods

In this notebook you will learn

* How to create a monitored `simpy.Resource`
* How to use the observer pattern to calculate running totals of metrics during a simulation run.

In [1]:
import simpy
import numpy as np
import itertools
import pandas as pd

simpy.__version__

'4.0.1'

# Monitoring a resource

A `simpy.Resource` is a python class.  You can inherit from this class and override the `request()` and `release()` methods.  This enables you to record the historical number of processes queuing for a resource and in progress OR calculate a online statistic such as the mean or variance in an event driven approach.

The class `MonitoredResource` inherits from `simpy.Resource`.  The methods `__init__`, `request()` and `release()` are overriden in a simple way.  

The mean number in the queue is updated each time a resource is requested using the formula

$L_q^t = \dfrac{n-1}{n}L_{q}^{t-1} + \dfrac{1}{n}q_t$

You can rearange this formula to one simpler to code in Python:

$L_q^t = L_{q}^{t-1} + \dfrac{\left(q_t -  L_{q}^{t-1}\right)}{n}$

If we define the variable `mean_x` as the running mean, `x` as a new observation, and `n` as the number of observations then in python the above formula is simply

```python
mean_x += (x - mean_x) / n
```

In [2]:
class MonitoredResource(simpy.Resource):
    '''
    adapted from: 
    https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html
    '''
    def __init__(self, *args, **kwargs):
        #super() is the super class i.e. simpy.Resource 
        super().__init__(*args, **kwargs)
        
        #the number of arrivals to the resource
        self.n = 0
        
        #the mean number queuing for service
        self.mean_queue = 0.0
        
        #mean service
        self.mean_in_service = 0.0
        
    def request(self, *args, **kwargs):
        
        #new arrival 
        self.n += 1
        
        #update the mean queue length
        self.mean_queue += (len(self.queue) - self.mean_queue) / self.n

        #update the mean in service
        self.mean_in_service += (self.count - self.mean_in_service) / self.n
        
        return super().request(*args, **kwargs)

## Parameters: 111 Model with Nurse call back Model 

In [3]:
# model parameters
RUN_LENGTH = 1000
N_OPERATORS = 13
N_NURSES = 9

ARRIVAL_RATE = 100
MEAN_IAT = 60 / ARRIVAL_RATE
CALL_LOW = 5
CALL_HIGH = 10
CALL_MODE = 7

#Should we show a trace of simulated events?
TRACE = False

#PRNG seeds (set these = None to get different runs)
ARRIVAL_SEED = 42
CALL_SEED = 101
CALLBACK_SEED = 1966
NURSE_SEED = 2020

## Distributions for the model

In [4]:

class Bernoulli():
    '''
    Convenience class for the Bernoulli distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, p, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        p: float
            probability of drawing a 1
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rand = np.random.default_rng(seed=random_seed)
        self.p = p
        
    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rand.binomial(n=1, p=self.p, size=size)
    

class Uniform():
    '''
    Convenience class for the Bernoulli distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, low, high, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        low: float
            lower range of the uniform
            
        high: float
            upper range of the uniform
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rand = np.random.default_rng(seed=random_seed)
        self.low = low
        self.high = high
        
    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rand.uniform(low=self.low, high=self.high, size=size)
    
    
class Exponential():
    '''
    Convenience class for the exponential distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, mean, random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        mean: float
            The mean of the exponential distribution
        
        random_seed: int, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        '''
        self.rand = np.random.default_rng(seed=random_seed)
        self.mean = mean
        
    def sample(self, size=None):
        '''
        Generate a sample from the exponential distribution
        
        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.
        '''
        return self.rand.exponential(self.mean, size=size)
    

class Triangular():
    '''
    Convenience class for the triangular distribution.
    packages up distribution parameters, seed and random generator.
    '''
    def __init__(self, low, mode, high, random_seed=None):
        self.rand = np.random.default_rng(seed=random_seed)
        self.low = low
        self.high = high
        self.mode = mode
        
    def sample(self, size=None):
        return self.rand.triangular(self.low, self.mode, self.high, size=size)

## Model classes

In [5]:
class Scenario(object):
    '''
    Parameter class for 111 simulation model
    '''
    def __init__(self):
        '''
        The init method sets up our defaults. 
        '''
        self.arrival_dist = Exponential(MEAN_IAT, random_seed=ARRIVAL_SEED)
        self.call_dist = Triangular(CALL_LOW, CALL_MODE, CALL_HIGH, 
                                    random_seed=CALL_SEED)
        self.nurse_dist = Uniform(10, 20, random_seed=NURSE_SEED)
        self.callback_dist = Bernoulli(p=0.4, random_seed=CALLBACK_SEED)

In [6]:
def trace(msg):
    '''
    Turing printing of events on and off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

In [7]:
class Patient(object):
    '''
    Encapsulates the process a patient caller undergoes when they dial 111
    and speaks to an operator who triages their call.
    '''
    def __init__(self, identifier, env, args):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        args: Scenario
            The input data for the scenario
        '''
        self.identifier = identifier
        self.env = env
        
        self.operators = args.operators
        self.call_dist = args.call_dist
        
        ##### MODIFICATION - passed in by args container class ################
        self.nurses = args.nurses
        self.nurse_dist = args.nurse_dist
        self.callback_dist = args.callback_dist
        #######################################################################
        
        ##### MODIFICATION - to track a callback ################
        self.callback = False
        self.waiting_time_nurse = 0.0
        self.nurse_call_duration = 0.0
        ##### MODIFICATION - to track a callback ################

    def service(self):
        '''
        simualtes the service process for a call operator
        
        1. request and wait for a call operator
        2. phone triage (triangular)
        3. exit system
        '''
        #record the time that call entered the queue
        start_wait = self.env.now

        #request an operator 
        with self.operators.request() as req:
            yield req
            
            #record the waiting time for call to be answered
            self.waiting_time = self.env.now - start_wait
            trace(f'operator answered call {self.identifier} at {self.env.now:.3f}')
            
            #sample call duration.
            self.call_duration = self.call_dist.sample()
            yield self.env.timeout(self.call_duration)
            
            self.operator_service_complete()
            
    
        ###### MODIFICATION - NURSE CALLBACK ##############################
        self.callback = self.callback_dist.sample()
        
        if self.callback:
            
            #record time starting to wait for nurse
            start_wait_nurse = self.env.now
            
            with self.nurses.request() as req:
                yield req
                
                #record the waiting time for nurse
                self.waiting_time_nurse = self.env.now - start_wait_nurse
                trace(f'nurse callback {self.identifier} at {self.env.now:.3f}')
                
                self.nurse_call_duration = self.nurse_dist.sample()
                yield self.env.timeout(self.nurse_call_duration)
                
                self.nurse_service_complete()
                
        ######################################################################
        
    def operator_service_complete(self):
        trace(f'call {self.identifier} ended {self.env.now:.3f}; waiting time was {self.waiting_time:.3f}')
    
    def nurse_service_complete(self):
        trace(f'nurse call {self.identifier} ended {self.env.now:.3f}')
        

In [8]:
#EXAMPLE ANSWER...
class UrgentCareCallCentre(object):
    def __init__(self, env, args):
        self.env = env
        self.args = args 
        
        ### MODIFICATION - list for storing patients #######
        self.patients = []
        ####################################################
        
    def arrivals_generator(self):
        '''
        IAT is exponentially distributed
        '''
        for caller_count in itertools.count(start=1):
            
            inter_arrival_time = self.args.arrival_dist.sample()
            yield env.timeout(inter_arrival_time)
            
            trace(f'call {caller_count} arrives at: {env.now:.3f}')
            
            new_caller = Patient(caller_count, self.env, self.args)
                        
            ############ MODIFICATION - store a Patient ref ##################
            self.patients.append(new_caller)
            ##################################################################
            
            env.process(new_caller.service())

# EXECUTE MODEL

In [9]:
#create simpy environment
env = simpy.Environment()
args = Scenario()

###### Monitored Resources ##################################
args.operators = MonitoredResource(env, capacity=N_OPERATORS)
args.nurses = MonitoredResource(env, capacity=N_NURSES)
##########################################################################

model = UrgentCareCallCentre(env, args)

env.process(model.arrivals_generator())
env.run(until=RUN_LENGTH)

print(f'end of run. simulation clock time = {env.now}')
print('\nSingle run results\n-------------------')

##### PRINT RESULTS
print('OPERATORS')
print(f'Mean queue length: {args.operators.mean_queue:.2f}')
print(f'Mean no. patients in service: {args.operators.mean_in_service:.2f}')

print('\n-------------------\nNURSES')
print(f'Mean queue length: {args.nurses.mean_queue:.2f}')
print(f'Mean no. patients in service: {args.nurses.mean_in_service:.2f}')

end of run. simulation clock time = 1000

Single run results
-------------------
OPERATORS
Mean queue length: 2.64
Mean no. patients in service: 12.00

-------------------
NURSES
Mean queue length: 33.89
Mean no. patients in service: 8.92


# Using any event in a process to trigger an audit

We can extend the above approach to create `MonitoredPatient`.  A key difference from `MonitoredResource` is that we will have **many** patients (as opposed to a single resource object).  To handle this we will use the **observer design pattern**.  We will refactor `Auditor` to register as an observer of the `MonitoredPatient` instances.  Each time a patient call with an operator is completed we will update the waiting time for an operator.

## MonitoredPatient

The class inherits from Patient and overrides the `operator_service_complete()` and `nurse_service_complete()` methods.  

We will create two new methods `register_observer(observer)` and `notify_observers()`.  The first of these methods allows us to update a list of observers (i.e. a patient could be monitored by multiple observers).  The second of these methods will loop through the list of observers and pass them a message.  The message will contain information about the patient (in fact we will just pass the patient object) and the type of event that has occured.  

Here is our monitored patient class:

In [10]:
class MonitoredPatient(Patient):
    '''
    Monitor a Patient.  Inherits from Patient
    Implemented using the observer design pattern
    
    A MonitoredPatient notifies its observers that a patient
    process has reached an event 
    1. completing call service
    2. completing nurse service
    '''
    def __init__(self, identifier, env, args, auditor):
        '''
        Constructor
        
        Params:
        -------
        patient: Patient
            patient process to monitor
            
        auditor: Auditor
            auditor
        '''
        super().__init__(identifier, env, args)
        self._observers = [auditor]
        
    def register_observer(self, observer):
        self._observers.append(observer)
    
    def notify_observers(self, *args, **kwargs):
        for observer in self._observers: 
            observer.process_event(*args, **kwargs)
    
    def operator_service_complete(self):
        #call the patients operator_service_complete method to execute logic
        super().operator_service_complete()
        
        #passes the patient (self) and a message
        self.notify_observers(self, 'operator_service_complete')
            
    def nurse_service_complete(self):
        #call the patients nurse_service_complete method to execute logic
        super().nurse_service_complete()
        
        #passes the patient (self) and a message
        self.notify_observers(self, 'nurse_service_complete')

Here is a refactored `Auditor` the new method is `process_event(*args, **kwargs)` that accepts a list and dict of arguments respectively.

The mean waiting times are calculated as running statistics.

In [11]:
class Auditor(object):
    '''
    Audits the simulation model when key events are triggered
    
    To simplify the illustration the functionality to schedule audits has been 
    removed.
    '''
    def __init__(self,):
        '''
        Auditor Constructor.
        '''
        #dict to hold states
        self.metrics = {}
        
        ##### MODIFICATION -running calcs that ###########################
        self.metrics['n'] = {'ops':0, 'nurse':0}
        self.metrics['mean_wait'] = {'ops':0.0, 'nurse':0.0}
        self.metrics['total_time'] = {'ops':0.0, 'nurse':0.0}
        ##################################################################

            
    ##### MODIFICATION - an event has occured in a process ##################        
    def process_event(self, *args, **kwargs):
        '''
        Running calculates each time a Patient process ends
        (when a patient departs the simulation model)
        
        Params:
        --------
        *args: list
            variable number of arguments. This is useful in case you need to
            pass different information for different events
        
        *kwargs: dict
            keyword arguments.  Same as args, but you can is a dict so you can
            use keyword to identify arguments.
        
        '''
        patient = args[0]
        msg = args[1]
        
        #there are cleaner ways of implementing this, but 
        #for simplicity it is implemented as an if-then statement
        if msg == 'operator_service_complete':
            self.metrics['n'] ['ops'] += 1
            n = self.metrics['n'] ['ops']
            
            #running calculation
            self.metrics['mean_wait']['ops'] += \
                (patient.waiting_time - self.metrics['mean_wait']['ops']) / n
            
        
        elif msg == 'nurse_service_complete':
            self.metrics['n'] ['nurse'] += 1
            n = self.metrics['n'] ['nurse']
            
            #running calculation
            self.metrics['mean_wait']['nurse'] += \
                (patient.waiting_time - self.metrics['mean_wait']['nurse']) / n
            
        
    #####################################################################

Finally here is a refactored `UrgentCareCallCentre`  The main difference is that we are now required to store a ref to an instance of `Auditor` in args and register this as an observer of each new `MonitoredPatient`.

In [12]:
class UrgentCareCallCentre(object):
    def __init__(self, env, args):
        self.env = env
        self.args = args 
        
    def arrivals_generator(self):
        '''
        IAT is exponentially distributed
        '''
        for caller_count in itertools.count(start=1):
            
            inter_arrival_time = self.args.arrival_dist.sample()
            yield env.timeout(inter_arrival_time)
            
            trace(f'call {caller_count} arrives at: {env.now:.3f}')
            
            ############ MODIFICATION - MonitoredPatient ######################
            new_caller = MonitoredPatient(caller_count, self.env, self.args,
                                          args.auditor)
             ##################################################################           
                        
            env.process(new_caller.service())

# EXECUTE MODEL

In [13]:
#create simpy environment
env = simpy.Environment()
args = Scenario()
args.auditor = Auditor()

#just for clarify - we are using standard resources
args.operators = simpy.Resource(env, capacity=N_OPERATORS)
args.nurses = simpy.Resource(env, capacity=N_NURSES)

model = UrgentCareCallCentre(env, args)

env.process(model.arrivals_generator())
env.run(until=RUN_LENGTH)

print(f'end of run. simulation clock time = {env.now}')
print('\nSingle run results\n-------------------')

##### PRINT RESULTS
metrics = args.auditor.metrics
print('OPERATORS')
print(f"Mean waiting time: {metrics['mean_wait']['ops']:.2f}")

print('\n-------------------\nNURSES')
print(f"Mean waiting time: {metrics['mean_wait']['nurse']:.2f}")

end of run. simulation clock time = 1000

Single run results
-------------------
OPERATORS
Mean waiting time: 1.68

-------------------
NURSES
Mean waiting time: 1.66
